In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint

from preprocessing import prepare_notes

# First simple model to get MVP

In [7]:
VOCAB = 2508

In [8]:
def build_model(lstm_input, model_output):
    '''
    Build and compile the model
    
    lstm_input: lstm_input.shape[1] = number of steps, lstm_input.shape[2] = number features needed for
    Bi-directional
    
    model_output: number of categories to output for classification
    
    returns: compiled model
    '''
    lstm_input, lstm_output = prepare_notes(notes)
    
    model = Sequential()
    forward_layer = LSTM(512, return_sequences=True)
    backward_layer = LSTM(512, activation='sigmoid', return_sequences=True,
                       go_backwards=True)
    model.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                         input_shape=(lstm_input.shape[1], lstm_input.shape[2])))
    model.add(Dense(VOCAB))
    
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    model
    
    return model



In [11]:
def fit_model(model, lstm_input, lstm_output):
    '''
    fit model and save checkpoint
    
    model: lstm model from function build_model()
    lstm_input: input to the model- output from function prepare_notes() 
    lstm_output: targets for model- output from function prepare_notes()
    
    returns: None
    '''
    # checkpoint
    filepath="checkpoint/weights-{epoch:02d}-{val_accuracy:.2f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    model.fit(lstm_input, lstm_output, epochs=200, batch_size=64, callbacks=callbacks_list)